In [ ]:
import qsharp
import qsharp_widgets
from qsharp._simulation import run_qir_gpu
print(qsharp.init(target_profile=qsharp.TargetProfile.Adaptive_RIF))

import cirq
print( f"Cirq version: {cirq.__version__}")

import qiskit
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
print( f"Qiskit version: {qiskit.__version__}")


number_of_shots = 2048

In [ ]:
%%qsharp

operation RunIsing() : Result[] {
    IsingModel1DEvolution(11, 1.0, 0.7, 4.0, 7)
}

operation IsingModel1DEvolution(
    N : Int,
    J : Double,
    g : Double,
    evolutionTime : Double,
    numberOfSteps : Int
) : Result[] {
    use qubits = Qubit[N];
    let dt : Double = evolutionTime / Std.Convert.IntAsDouble(numberOfSteps);
    let theta_x = - g * dt;
    let theta_zz = J * dt;

    for i in 1..numberOfSteps {
        for q in qubits {
            Rx(2.0 * theta_x, q);
        }
        for j in 0..2..N-2 {
            Rzz(2.0 * theta_zz, qubits[j], qubits[j + 1]);
        }
        for j in 1..2..N-2 {
            Rzz(2.0 * theta_zz, qubits[j], qubits[j + 1]);
        }
    }

    MResetEachZ(qubits)
}

In [ ]:
from qsharp.code import RunIsing
 # Only do 500 shots for sparse simulator, otherwise it is too slow
results = qsharp.run(RunIsing, 2048)
bitstrings = [
    "".join("1" if bit == qsharp.Result.One else "0" for bit in measurement)
    for measurement in results
]
qsharp_widgets.Histogram(bitstrings, items = "top-25", sort="high-to-low")

In [ ]:
from qsharp.code import IsingModel1DEvolution
qir = qsharp.compile(IsingModel1DEvolution, 21, 1.0, 0.7, 4.0, 7)
print(f"Compiled to {len(str(qir))} QIR lines.")
print( f"Running: {number_of_shots} shots using QDK GPU simulator.")
gpu_results = run_qir_gpu(qir, shots=number_of_shots, noise=None, seed=None)
gpu_bitstrings = [
    "".join("1" if bit == qsharp.Result.One else "0" for bit in measurement)
    for measurement in gpu_results
]
qsharp_widgets.Histogram(gpu_bitstrings, items = "top-25", sort="high-to-low")


In [ ]:
from math import pi
from typing import List

def get_cirq_circuit(
    N: int,
    J: float,
    g: float,
    evolutionTime: float,
    numberOfSteps: int
) -> cirq.Circuit:
    # Allocate qubit grid
    qubits = cirq.LineQubit.range(N)

    # Compute the time step
    dt: float = evolutionTime / numberOfSteps

    theta_x = -g * dt
    Rx = cirq.rx(2.0 * theta_x)
    theta_zz = J * dt
    Rzz = cirq.ZZPowGate(exponent = 2.0 * theta_zz/pi, global_shift = -0.5)

    operations: List[cirq.Operation] = []

    for _ in range(numberOfSteps):
        for qubit in qubits:
            operations.append(Rx.on(qubit))

        for j in range(0, N - 1, 2):
            operations.append(Rzz.on(qubits[j], qubits[j + 1]))

        for j in range(1, N - 1, 2):
            operations.append(Rzz.on(qubits[j], qubits[j + 1]))

    operations.append(cirq.measure(*qubits, key="m"))

    return cirq.Circuit(operations)


In [ ]:
ising_circuit = get_cirq_circuit(11, 1.0, 0.7, 4.0, 7)

simulator = cirq.Simulator()
print( f"Running: {number_of_shots} shots using Cirq.")
cirq_results = simulator.run(ising_circuit, repetitions=number_of_shots)
measurements = cirq_results.measurements["m"]
cirq_bitstrings = [
    "".join("1" if bit else "0" for bit in outcome)
    for outcome in measurements
]
qsharp_widgets.Histogram(cirq_bitstrings, items = "top-25", sort="high-to-low")

In [ ]:
def get_qiskit_circuit(
    N: int,
    J: float,
    g: float,
    evolutionTime: float,
    numberOfSteps: int
) -> qiskit.QuantumCircuit:

    qc = qiskit.QuantumCircuit(N, N)

    dt: float = evolutionTime / numberOfSteps
    theta_x = -g * dt
    theta_zz = J * dt

    for _ in range(numberOfSteps):
        for qubit in range(N):
            qc.rx(2.0 * theta_x, qubit)

        for j in range(0, N - 1, 2):
            qc.rzz(2.0 * theta_zz, j, j + 1)

        for j in range(1, N - 1, 2):
            qc.rzz(2.0 * theta_zz, j, j + 1)

    qc.measure(range(N), range(N))

    return qc


In [ ]:
qc = get_qiskit_circuit(11, 1.0, 0.7, 4.0, 7)

backend = AerSimulator(method="statevector")
compiled = transpile(qc, backend)
job = backend.run(compiled, shots=number_of_shots, memory=True)
result = job.result()
qiskit_results = result.get_memory()
qsharp_widgets.Histogram(qiskit_results, items = "top-25", sort="high-to-low")